In [2]:

#Load libraries
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import torch.nn as nn
import torch.nn.functional as F


In [3]:
torch.cuda.empty_cache()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -u "/content/drive/Shareddrives/Research/Mominul/Data_BanglaLekha_Isolated.zip" -d "/content/drive/Shareddrives/Research/Mominul/Extracted_Database"

In [ ]:
#checking for device to Get cpu or gpu for training.

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


num_classes = 10
num_epochs = 20
batch_size = 128
learning_rate = 0.001

#Transforms
transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])


train_path = '/content/drive/MyDrive/Thesis/SampleDataSet/Extracted_Database/Data_BanglaLekha_Isolated/data/train_set'
test_path = '/content/drive/MyDrive/Thesis/SampleDataSet/Extracted_Database/Data_BanglaLekha_Isolated/data/validation_set'


In [ ]:
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
valid_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=64, shuffle=True
)

#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

print(classes)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, input_channel, num_classes=10):
        super().__init__()
        self.conv1 = nn.Sequential(
            # transforming (bsize x 1 x 224 x 224) to (bsize x 96 x 54 x 54)
            # From floor((n_h - k_s + p + s)/s), floor((224 - 11 + 3 + 4) / 4) => floor(219/4) => floor(55.5) => 55
            nn.Conv2d(input_channel, 96, kernel_size=11, stride=4, padding=3),  # (batch_size * 96 * 55 * 55)
            nn.ReLU(inplace=True),  # (batch_size * 96 * 55 * 55)
            # nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2))  # (batch_size * 96 * 27 * 27)
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, padding=2),  # (batch_size * 256 * 27 * 27)
            nn.ReLU(inplace=True),
            # nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2))  # (batch_size * 256 * 13 * 13)
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, padding=1),  # (batch_size * 384 * 13 * 13)
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # (batch_size * 384 * 13 * 13)
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # (batch_size * 256 * 13 * 13)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (batch_size * 256 * 6 * 6)
            nn.Flatten())  # (batch_size * 9216)
        self.fc = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),  # (batch_size * 4096)
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),  # (batch_size * 4096)
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes))  # (batch_size * 10)

        self.conv1.apply(self.init_weights)
        self.conv2.apply(self.init_weights)
        self.conv3.apply(self.init_weights)
        self.fc.apply(self.init_weights)

    def init_weights(self, layer):
        if type(layer) == nn.Linear or type(layer) == nn.Conv2d:
            nn.init.xavier_uniform_(layer.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.fc(out)

        return out


In [ ]:
model = AlexNet(input_channel=3, num_classes=10).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(15600, 100 * correct / total)) 


    #testing
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(4148, 100 * correct / total))